# VARify


## Workflow


<img src="wf.png" width="600">

## Exploring Data

In [244]:
import pandas as pd

# mpileup 
mp = pd.read_table('../data/test/mpiletest-out.txt', header=None)
mp = mp.rename(columns={1:"snp_pos", 4:"pileup"})

# custom out file
cf = pd.read_table('../data/test/snp_out.txt')

In [245]:
# mpileup file
mp

,0,snp_pos,2,3,pileup,5,6
0,seq1,521,G,26,".,......C.T.A,..,..,.,.,,^].",DJJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]]
1,seq1,522,T,25,",...........,..,..,.,.,,.",JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
2,seq1,523,A,25,tTTTTTTTTCTTtTTtTCtTtTttC,JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
3,seq1,524,C,27,",...G.......,.G,..,.,.,,A..",JJJJJJJJJJJJJJJJJJJJJJJJG88,]]]]]]]]]]]]]]]]]]]]]]]]]]]


In [246]:
# column names
# cols = ['chr_id', 'snp_pos', 'ref_allele', 
#         'alt_allele', 'gene_id', 'mrna_id',
#         'prot_id', 'strand', 'effect', 
#         'snp_cds_pos', 'codon1_genome_pos',
#         'codon2_genome_pos', 'codon3_genome_pos', 
#         'snp_aa_pos', 'ref_codon','alt_codon', 
#         'ref_aa', 'alt_aa']

sub_cols = ['chr_id', 'snp_pos', 'ref_allele', 
            'alt_allele', 'codon1_genome_pos',
            'codon2_genome_pos', 'codon3_genome_pos', 
            'ref_codon']

# subset of custom file, cf
sub_cf = cf[sub_cols]
sub_cf

,chr_id,snp_pos,ref_allele,alt_allele,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,ref_codon
0,chr1,521,G,C,519,520,521,AAG
1,chr1,523,A,T,521,522,523,GTA
2,chr1,524,C,G,522,523,524,TAC


## Merging the two dataframes together

In [247]:
sub_mp = mp[['snp_pos','pileup']]
merged_var = pd.merge(cf, sub_mp, on='snp_pos')
merged_var

,chr_id,snp_pos,ref_allele,alt_allele,gene_id,mrna_id,prot_id,strand,effect,snp_cds_pos,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,snp_aa_pos,ref_codon,alt_codon,ref_aa,alt_aa,pileup
0,chr1,521,G,C,seq1,seq1.1,seq1.1,+,synonymous,265,519,520,521,89,AAG,AAC,K,N,".,......C.T.A,..,..,.,.,,^]."
1,chr1,523,A,T,seq1,seq1.1,seq1.1,+,synonymous,267,521,522,523,89,GTA,GTT,E,E,tTTTTTTTTCTTtTTtTCtTtTttC
2,chr1,524,C,G,seq1,seq1.1,seq1.1,+,synonymous,268,522,523,524,89,TAC,TAG,Y,X,",...G.......,.G,..,.,.,,A.."


## Split Dataframe

Splitting the snpeff table by codons, get genome position and alt/ref codon



In [249]:
# How to use apply
# pd.DataFrame.apply(test_df, funct)

# For every row in here, we're going to apply this uniqueSNP function
def varOnly(a):
    
    x = [i for i in a if i.upper() in ['A','C','T','G']]
    
    return x

def uniqueSNP(x):
    l = str(x['pileup'])
    # make upper
    l = l.upper() # just to make simple
    l = varOnly(l)

    # Get list of unique
    a = [i for i in l]
    a_unique = list(set(a)) 
    
    if len(a_unique) > 2: 
        print(x['snp_pos'], 'has > 2 SNPS.')
    
    # create a dictionary
    df = pd.DataFrame()
    snp_dict = {}
    temp = [i for i in x['ref_codon']]

    for snp in a_unique:
        snp_dict['snp_pos'] = int(x['snp_pos']) # this would need to change
        snp_dict['varify_alt'] = snp

        for i in range(len(temp)):
            temp[i] = snp     
            snp_dict[f"varify_codon{i+1}"] = ''.join(temp)
            temp = [i for i in x['ref_codon']]

        df = df.append(snp_dict, ignore_index=True)
        
        snp_dict = {}

    df = pd.merge(sub_cf, df, on='snp_pos')
    return df


test = merged_var.apply(axis=1, func=uniqueSNP)
pd.concat(list(test))

521 has > 2 SNPS.


,chr_id,snp_pos,ref_allele,alt_allele,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,ref_codon,varify_alt,varify_codon1,varify_codon2,varify_codon3
0,chr1,521,G,C,519,520,521,AAG,A,AAG,AAG,AAA
1,chr1,521,G,C,519,520,521,AAG,T,TAG,ATG,AAT
2,chr1,521,G,C,519,520,521,AAG,C,CAG,ACG,AAC
0,chr1,523,A,T,521,522,523,GTA,T,TTA,GTA,GTT
1,chr1,523,A,T,521,522,523,GTA,C,CTA,GCA,GTC
0,chr1,524,C,G,522,523,524,TAC,A,AAC,TAC,TAA
1,chr1,524,C,G,522,523,524,TAC,G,GAC,TGC,TAG


### To Do
* [x] output comparison
* [ ] Type Effect
* [x] Make into Function
* [ ] Build larger test data
* [ ] Nextflow with Assembly/Sam to mpileup


### Notes
**Test Data:**
* Take processed mRNA transcript (FASTA) —> VCF w/ 2-3 SNPS

**Process:** 
* RNA FASTA -align to genome using gmap->SAM/BAM—>BAM -variantcalling-> VCF —> Capture Variants
* DNA FASTA -align to genome using gmap->SAM/BAM—>BAM -variantcalling-> VCF —> Capture Variants |—> Compare RV and GV

In [ ]:
top_snp = majorSNP(merged_var)
top_strand = strand(top_snp)

In [ ]:
snp_dict = {'confirmed_snp' : top_snp, 'confirmed_strand' : strand}
merged_var['confirmed_snp'] = top_snp
merged_var['confirmed_strand'] = top_strand

# proposed final output
merged_var.drop(columns='pileup')

In [ ]:
# majority snp
def split(word):
    return list(word)

def majorSNP(merged_var):
    n=0
    arr = split(merged_var['pileup'][n])
    
    snps = pd.Series(arr).drop_duplicates().tolist()
    
    count = dict.fromkeys(snps, 0)

    for snp_key in snps:
        for arr_key in arr: 
            if snp_key == arr_key: 
                count[snp_key] = count.get(snp_key, 0) + 1
                
    top_snp = sorted(count)[0]
    return top_snp

# strand
def strand(x):
    pos = ['A','C','T','G', 'N']
    rev = ['a','c','t','g', 'n']
    if x in pos:
        return '+'
    if x in rev:
        return '-'
    
# grab all snps
#def uniqueSNP():
    # get the list of SNPs
    
    # split into unique
    

def combine(x):
    return print(*x, sep='')

    